In [1]:
import numpy as np
import pandas as pd
import bcolz
import time
import logging
import datetime

import sys
sys.path.append('..')

from bcolzutils import *
from util import *

import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, LearningRateScheduler
from keras import optimizers
from keras.regularizers import l2 

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input


import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

Using TensorFlow backend.
/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
arch = "vgg19"

percent = 0.25
#percent = 1
epochs=15
num_classes = 133
batch_size = 64
lr=1e-3
momentum=0.9
weight_decay = 1e-5
test_prefix=""

def lr_schedule(epoch):
    """ divides the lr by 10 every 5 epochs"""
    n = epoch // 5
    return lr * (0.1 ** n)

if percent < 1:
    test_prefix = "_test"
    
test_result = f'bottleneck_features_{arch}_result{test_prefix}.npz'
model_path = f'../saved_models/weights.best.topmodel.{arch}{test_prefix}.hdf5'
loss_history_csv_name = f'train_top_model_{arch}_loss_history{test_prefix}.csv'

d = datetime.datetime.today()

logging.basicConfig(level='DEBUG',
                    handlers=[
                              logging.StreamHandler()])
log = logging.getLogger(__name__)

basedir="/home/tutysara/src/myprojects/dog-project/dogImages"

train_idx_path = basedir+ "/train_list.txt"
valid_idx_path = basedir+ "/valid_list.txt"
test_idx_path = basedir+ "/test_list.txt"

train_name = basedir + '/train_data'
valid_name = basedir + '/valid_data'
test_name = basedir + '/test_data'

In [3]:
%ls -l {basedir}

total 532
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_test_data.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_test_labels.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:12 bottleneck_features_vgg19_test_y_pred.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:12 bottleneck_features_vgg19_test_y_true.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_train_data.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_train_labels.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_valid_data.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 20:00 bottleneck_features_vgg19_valid_labels.bclz/
drwxr-xr-x 135 tutysara tutysara   4096 Mar 27  2017 test/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 19:52 test_data_data.bclz/
drwxrwxr-x   4 tutysara tutysara   4096 Mar  2 19:52 test_data_labels.bclz/
-rw-rw-

In [ ]:
# alternate image loading
# define function to load train, test, and validation datasets
from sklearn.datasets import load_files 
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
valid_files, valid_targets = load_dataset(basedir + '/../' +'dogImages/valid')
test_files, test_targets = load_dataset(basedir + '/../' +'dogImages/test')
train_files, train_targets = load_dataset(basedir + '/../' +'dogImages/train')

In [ ]:
import random
indices = random.sample(range(len(train_files)), 15)

In [ ]:
print(train_files[indices])
print(np.argmax(train_targets[indices], axis=1))

In [ ]:
valid_data_size = int(valid_files.shape[0]*percent)
test_data_size = int(test_files.shape[0]*percent)
train_data_size = int(train_files.shape[0]*percent)

if percent < 1:
    valid_files = valid_files[:valid_data_size]
    valid_targets = valid_targets[:valid_data_size]
    
    test_files = test_files[:test_data_size]
    test_targets = test_targets[:test_data_size]
    
    train_files = train_files[:train_data_size]
    train_targets = train_targets[:train_data_size]

In [ ]:
print(valid_files.shape, valid_targets.shape)
print(test_files.shape, test_targets.shape)
print(train_files.shape, train_targets.shape)

In [ ]:
# convert and load images
from keras.preprocessing import image                  
from tqdm import tqdm
# load and preprocess data
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

# pre-process the data for Keras
valid_tensors = paths_to_tensor(valid_files).astype('float32')
test_tensors = paths_to_tensor(test_files).astype('float32')
train_tensors = paths_to_tensor(train_files).astype('float32')

In [ ]:
valid_data = vgg19_preprocess_input(valid_tensors, mode='caffe')
test_data = vgg19_preprocess_input(test_tensors, mode='caffe')
train_data = vgg19_preprocess_input(train_tensors, mode='caffe')

In [ ]:
# Generate a model with all layers (with top)
vgg19 = VGG19(weights='imagenet', include_top=True)

#Add a layer where input is the output of the  second last layer 
x = Dense(num_classes, activation='softmax', name='my_predictions')(vgg19.layers[-2].output)

for layer in vgg19.layers:
    layer.trainable = False
    
#Then create the corresponding model 
my_model = Model(input=vgg19.input, output=x)
my_model.layers[-3].trainable = True
my_model.layers[-2].trainable = True
my_model.layers[-1].trainable = True
#my_model.summary()

In [ ]:
for layer in my_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])
my_model.fit( train_data, train_targets,
          epochs=epochs,
          validation_data=(valid_data, valid_targets),
          callbacks=[early_stopping, lrscheduler])

In [ ]:
#2 read data and convert to tensor
col_names = ["X", "y"]
train_data_df = pd.read_csv(train_idx_path, sep=" ", header=None, names=col_names)
valid_data_df = pd.read_csv(valid_idx_path, sep=" ", header=None, names=col_names)
test_data_df = pd.read_csv(test_idx_path, sep=" ", header=None, names=col_names)

train_data_df = train_data_df[:int(train_data_df.shape[0]*percent)]
valid_data_df = valid_data_df[:int(valid_data_df.shape[0]*percent)]
test_data_df = test_data_df[:int(test_data_df.shape[0]*percent)]

train_data_df.y = train_data_df.y-1
valid_data_df.y = valid_data_df.y-1
test_data_df.y = test_data_df.y-1

valid_files = valid_data_df.X.apply(lambda x: basedir+"/"+x)
test_files = test_data_df.X.apply(lambda x: basedir+"/"+x)
train_files = train_data_df.X.apply(lambda x: basedir+"/"+x)

valid_labels = np_utils.to_categorical(valid_data_df.y, num_classes) 
test_labels = np_utils.to_categorical(test_data_df.y, num_classes)
train_labels = np_utils.to_categorical(train_data_df.y, num_classes)

In [ ]:
print(valid_files.shape, valid_labels.shape)
print(test_files.shape, test_labels.shape)
print(train_files.shape, train_labels.shape)

In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

my_model.fit(train_data, train_labels,
          epochs=epochs,
          validation_data=(valid_data, valid_labels),
          callbacks=[early_stopping])

In [4]:
#3 read bcolz data
bclz_valid_data = bcolz.carray(rootdir= valid_name+'_data.bclz', mode='r')
bclz_test_data = bcolz.carray(rootdir= test_name + '_data.bclz', mode='r')
bclz_train_data = bcolz.carray(rootdir= train_name+ '_data.bclz', mode='r')


bclz_valid_labels = bcolz.carray(rootdir= valid_name+'_labels.bclz', mode='r')
bclz_test_labels = bcolz.carray(rootdir= test_name + '_labels.bclz', mode='r')
bclz_train_labels = bcolz.carray(rootdir= train_name+ '_labels.bclz', mode='r')

print(bclz_valid_data.shape, bclz_valid_labels.shape)
print(bclz_test_data.shape, bclz_test_labels.shape) 
print(bclz_train_data.shape, bclz_train_labels.shape) 

(835, 224, 224, 3) (835, 133)
(836, 224, 224, 3) (836, 133)
(6680, 224, 224, 3) (6680, 133)


In [5]:
# Generate a model with all layers (with top)
vgg19 = VGG19(weights='imagenet', include_top=True)

#Add a layer where input is the output of the  second last layer 
x = Dense(num_classes, activation='softmax', name='my_predictions')(vgg19.layers[-2].output)

for layer in vgg19.layers:
    layer.trainable = False
    
#Then create the corresponding model 
my_model = Model(input=vgg19.input, output=x)
my_model.layers[-3].trainable = True
my_model.layers[-2].trainable = True
my_model.layers[-1].trainable = True
#my_model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[25088,4096]
	 [[Node: fc1/kernel/Assign = Assign[T=DT_FLOAT, _class=["loc:@fc1/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fc1/kernel, fc1/random_uniform)]]

Caused by op 'fc1/kernel/Assign', defined at:
  File "/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/tutysara/anaconda2/envs/dog-project/lib/python3.6/site-packages/ipykernel/kernelapp.py", l

In [ ]:
for layer in my_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
print(bclz_valid_data.shape, bclz_valid_labels.shape)
print(bclz_test_data.shape, bclz_test_labels.shape) 
print(bclz_train_data.shape, bclz_train_labels.shape) 

In [30]:
valid_gen =bcolz_data_generator(bclz_valid_data, bclz_valid_labels, batch_size=batch_size, preprocess=vgg19_preprocess_input)
test_gen =bcolz_data_generator(bclz_test_data, bclz_test_labels, batch_size=batch_size, preprocess=vgg19_preprocess_input)
train_gen =bcolz_data_generator(bclz_train_data, bclz_train_labels, batch_size=batch_size, preprocess=vgg19_preprocess_input)

In [41]:
tX, ty = next(train_gen)

In [42]:
# check inputs
print(tX.shape)
print(tX[1,34,5])
print(td.shape)
print(td[1,34,5])

(64, 224, 224, 3)
[ 77.06099701  38.22100067  25.31999969]
(6680, 224, 224, 3)
[ 77.06099701  38.22100067  25.31999969]


In [43]:
# check labels
print(ty.shape)
print(bclz_train_labels.shape)

print(np.argmax(ty[:15], axis=1))
print(np.argmax(bclz_train_labels[:15], axis=1))

(64, 133)
(6680, 133)
[74 74 74 74 74 74 74 74 74 74 74 74 74 74 74]
[74 74 74 74 74 74 74 74 74 74 74 74 74 74 74]


In [19]:
vd = vgg19_preprocess_input(bclz_valid_data[:bclz_valid_data.shape[0]])
tsd = vgg19_preprocess_input(bclz_test_data[:bclz_test_data.shape[0]])
td = vgg19_preprocess_input(bclz_train_data[:bclz_train_data.shape[0]])

In [23]:
print(vd.shape, bclz_valid_labels.shape)
print(tsd.shape, bclz_test_labels.shape)
print(td.shape, bclz_train_labels.shape)

(835, 224, 224, 3) (835, 133)
(836, 224, 224, 3) (836, 133)
(6680, 224, 224, 3) (6680, 133)


In [ ]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])

my_model.fit_generator(train_gen,
          steps_per_epoch= (1 + int(bclz_train_data.shape[0] // batch_size)),
          epochs=epochs,
          validation_data=valid_gen,
          validation_steps= (1 + int(bclz_valid_data.shape[0] // batch_size)),
          callbacks=[early_stopping, lrscheduler],
          use_multiprocessing=True )

Epoch 1/15


In [26]:
checkpointer = ModelCheckpoint(filepath=model_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
csv_logger = CSVLogger(loss_history_csv_name, append=True, separator=',')
lrscheduler = LearningRateScheduler(schedule=lr_schedule)

my_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=lr, momentum=momentum),
              metrics=['accuracy'])
my_model.fit( td, bclz_train_labels,
          epochs=epochs,
          validation_data=(vd, bclz_valid_labels),
          callbacks=[early_stopping, lrscheduler])

Train on 6680 samples, validate on 835 samples
Epoch 1/15
6680/6680 [==============================] - 63s 9ms/step - loss: 1.5875 - acc: 0.6268 - val_loss: 1.0389 - val_acc: 0.7090
Epoch 2/15
6680/6680 [==============================] - 62s 9ms/step - loss: 0.2750 - acc: 0.9226 - val_loss: 0.8788 - val_acc: 0.7497
Epoch 3/15
 736/6680 [==>...........................] - ETA: 50s - loss: 0.1179 - acc: 0.9647

KeyboardInterrupt: 